In [1]:
import os
import argparse
import gym
import numpy as np
from itertools import count

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical

In [3]:
is_cuda = torch.cuda.is_available()

In [4]:
args = {
    "gamma": 0.99,
    "decay_rate": 0.99,
    "learning_rate": 1e-4,
    "batch_size": 20,
    "seed": 87,
    "test": False
}

In [5]:
env = gym.make('Pong-v0')
env.seed(args["seed"])
torch.manual_seed(args["seed"])

In [6]:
D = 80 * 80
test = args["test"]
if test == True:
    render = True
else:
    render = False

In [7]:
def prepro(I):
    """ prepro 210x160x3 into 6400 """
    I = I[35:195]
    I = I[::2, ::2, 0]
    I[I == 144] = 0
    I[I == 109] = 0
    I[I != 0 ] = 1
    return I.astype(np.float).ravel()

In [8]:
class Policy(nn.Module):
    def __init__(self, num_actions=2):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(6400, 200)
        self.affine2 = nn.Linear(200, num_actions) # action 1: static, action 2: move up, action 3: move down
        self.num_actions = num_actions
        self.saved_log_probs = []
        self.rewards = []
        rand_var = torch.tensor([0.5,0.5])
        if is_cuda: rand_var = rand_var.cuda()
        self.random = Categorical(rand_var)

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)


    def select_action(self, x):
        x = Variable(torch.from_numpy(x).float().unsqueeze(0))
        if is_cuda: x = x.cuda()
        probs = self.forward(x)
        m = Categorical(probs)
        action = m.sample()

        self.saved_log_probs.append(m.log_prob(action))
        return action

In [9]:
# built policy network
policy = Policy()
if is_cuda:
    policy.cuda()

In [10]:
# check & load pretrain model
if os.path.isfile('pg_params.pkl'):
    print('Load Policy Network parametets ...')
    policy.load_state_dict(torch.load('pg_params.pkl'))

In [11]:
# construct a optimal function
optimizer = optim.RMSprop(policy.parameters(), lr=args["learning_rate"], weight_decay=args["decay_rate"])

In [12]:
def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + args["gamma"] * R
        rewards.insert(0, R)
    # turn rewards to pytorch tensor and standardize
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-6)

    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(- log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.stack(policy_loss).sum()
    if is_cuda:
        policy_loss.cuda()
    policy_loss.backward()
    optimizer.step()

    # clean rewards and saved_actions
    del policy.rewards[:]
    del policy.saved_log_probs[:]

In [ ]:
# Main loop
running_reward = None
reward_sum = 0
prev_x = None
for i_episode in count(1):
    state = env.reset()
    for t in range(10000):
        if render: env.render()
        cur_x = prepro(state)
        x = cur_x - prev_x if prev_x is not None else np.zeros(D)
        prev_x = cur_x
        action = policy.select_action(x)
        action_env = action + 2
        state, reward, done, _ = env.step(action_env)
        reward_sum += reward

        policy.rewards.append(reward)
        if done:
            # tracking log
            running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
            print('REINFORCE ep %03d done. reward: %f. reward running mean: %f' % (i_episode, reward_sum, running_reward))
            reward_sum = 0
            break


    # use policy gradient update model weights
    if i_episode % args["batch_size"] == 0:
        finish_episode()

    # Save model in every 50 episode
    if i_episode % 50 == 0:
        print('ep %d: model saving...' % (i_episode))
        torch.save(policy.state_dict(), 'pg_params.pkl')

REINFORCE ep 001 done. reward: -21.000000. reward running mean: -21.000000
REINFORCE ep 002 done. reward: -21.000000. reward running mean: -21.000000
REINFORCE ep 003 done. reward: -18.000000. reward running mean: -20.970000
REINFORCE ep 004 done. reward: -19.000000. reward running mean: -20.950300
REINFORCE ep 005 done. reward: -20.000000. reward running mean: -20.940797
REINFORCE ep 006 done. reward: -21.000000. reward running mean: -20.941389
REINFORCE ep 007 done. reward: -21.000000. reward running mean: -20.941975
REINFORCE ep 008 done. reward: -21.000000. reward running mean: -20.942555
REINFORCE ep 009 done. reward: -21.000000. reward running mean: -20.943130
REINFORCE ep 010 done. reward: -21.000000. reward running mean: -20.943699
REINFORCE ep 011 done. reward: -21.000000. reward running mean: -20.944262
REINFORCE ep 012 done. reward: -20.000000. reward running mean: -20.934819
REINFORCE ep 013 done. reward: -21.000000. reward running mean: -20.935471
REINFORCE ep 014 done. re